# Polynomial weight model model

In [1]:
using DynamicHMCModels

ProjDir = rel_path_d("..", "scripts", "04")
cd(ProjDir)

Import the dataset.

In [2]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults and standardize

In [3]:
df2 = filter(row -> row[:age] >= 18, df);
df2[:weight] = convert(Vector{Float64}, df2[:weight]);
df2[:weight_s] = (df2[:weight] .- mean(df2[:weight])) / std(df2[:weight]);
df2[:weight_s2] = df2[:weight_s] .^ 2;

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at string:2
└ @ Core string:2
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at string:2
└ @ Core string:2
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at string:3
└ @ Core string:3
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at string:3
└ @ Core string:3
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at string:3
└ @ Core string:3
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│   

Show the first six rows of the dataset.

In [4]:
first(df2, 6)

6×6 DataFrame
│ Row │ height  │ weight  │ age     │ male  │ weight_s  │ weight_s2 │
│     │ Float64 │ Float64 │ Float64 │ Int64 │ Float64   │ Float64   │
├─────┼─────────┼─────────┼─────────┼───────┼───────────┼───────────┤
│ 1   │ 151.765 │ 47.8256 │ 63.0    │ 1     │ 0.439097  │ 0.192806  │
│ 2   │ 139.7   │ 36.4858 │ 63.0    │ 0     │ -1.31718  │ 1.73498   │
│ 3   │ 136.525 │ 31.8648 │ 65.0    │ 0     │ -2.03287  │ 4.13256   │
│ 4   │ 156.845 │ 53.0419 │ 41.0    │ 1     │ 1.24699   │ 1.55498   │
│ 5   │ 145.415 │ 41.2769 │ 51.0    │ 0     │ -0.575156 │ 0.330804  │
│ 6   │ 163.83  │ 62.9926 │ 35.0    │ 1     │ 2.78812   │ 7.77364   │

Then define a structure to hold the data: observables, covariates, and the degrees of freedom for the prior.

Linear regression model ``y ∼ Xβ + ϵ``, where ``ϵ ∼ N(0, σ²)`` IID.

In [5]:
struct ConstraintHeightProblem{TY <: AbstractVector, TX <: AbstractMatrix}
    "Observations."
    y::TY
    "Covariates"
    X::TX
end;

Then make the type callable with the parameters *as a single argument*.

In [6]:
function (problem::ConstraintHeightProblem)(θ)
    @unpack y, X, = problem   # extract the data
    @unpack β, σ = θ            # works on the named tuple too
    ll = 0.0
    ll += logpdf(Normal(178, 100), X[1]) # a = X[1]
    ll += logpdf(Normal(0, 10), X[2]) # b1 = X[2]
    ll += logpdf(Normal(0, 10), X[3]) # b2 = X[3]
    ll += logpdf(TDist(1.0), σ)
    ll += loglikelihood(Normal(0, σ), y .- X*β)
    ll
end

Setup data and inits.

In [7]:
N = size(df2, 1)
X = hcat(ones(N), hcat(df2[:weight_s], df2[:weight_s2]));
y = convert(Vector{Float64}, df2[:height])
p = ConstraintHeightProblem(y, X);
p((β = [1.0, 2.0, 3.0], σ = 1.0))

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at string:2
└ @ Core string:2
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at string:2
└ @ Core string:2
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at string:3
└ @ Core string:3


-4.0013242576346947e6

Use a function to return the transformation (as it varies with the number of covariates).

In [8]:
problem_transformation(p::ConstraintHeightProblem) =
    as((β = as(Array, size(p.X, 2)), σ = asℝ₊))

problem_transformation (generic function with 1 method)

Wrap the problem with a transformation, then use Flux for the gradient.

In [9]:
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));

Draw samples.

In [10]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.0027 s/step ...done
MCMC, adapting ϵ (25 steps)
0.00068 s/step ...done
MCMC, adapting ϵ (50 steps)
0.00043 s/step ...done
MCMC, adapting ϵ (100 steps)
0.00044 s/step ...done
MCMC, adapting ϵ (200 steps)
0.00029 s/step ...done
MCMC, adapting ϵ (400 steps)
0.00021 s/step ...done
MCMC, adapting ϵ (50 steps)
0.00022 s/step ...done
MCMC (1000 steps)
0.00027 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [11]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));
posterior[1:5]

5-element Array{NamedTuple{(:β, :σ),Tuple{Array{Float64,1},Float64}},1}:
 (β = [154.8818073723645, 5.575696266837626, -0.1815330696129215], σ = 5.110839906687951) 
 (β = [154.76150681795374, 5.313886255519923, 0.06365709058054754], σ = 5.187591705683426)
 (β = [155.0223696865696, 5.296115496001354, 0.009422906312992675], σ = 5.477372297690153)
 (β = [155.39053999453046, 5.506729084206808, 0.03220722492707618], σ = 5.26135458255397) 
 (β = [153.888923851898, 5.922851028473373, 0.24700799314937727], σ = 5.022943032265271)  

Extract the parameter posterior means: `β`,

In [12]:
posterior_β = mean(first, posterior)

3-element Array{Float64,1}:
 154.6231386427075     
   5.856041468814334   
  -0.013078530905223562

then `σ`:

In [13]:
posterior_σ = mean(last, posterior)

5.094422864458341

Effective sample sizes (of untransformed draws)

In [14]:
ess = mapslices(effective_sample_size,
                get_position_matrix(chain); dims = 1)

1×4 Array{Float64,2}:
 1000.0  423.367  933.186  1000.0

NUTS-specific statistics

In [15]:
NUTS_statistics(chain)

cmdstan_result = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
           Mean         SD       Naive SE       MCSE      ESS
    a 154.609019750 0.36158389 0.0057171433 0.0071845548 1000
   b1   5.838431778 0.27920926 0.0044146860 0.0048693502 1000
   b2  -0.009985954 0.22897191 0.0036203637 0.0047224478 1000
sigma   5.110136300 0.19096315 0.0030193925 0.0030728192 1000

Quantiles:
          2.5%        25.0%        50.0%       75.0%        97.5%
    a 153.92392500 154.3567500 154.60700000 154.8502500 155.32100000
   b1   5.27846200   5.6493250   5.83991000   6.0276275   6.39728200
   b2  -0.45954687  -0.1668285  -0.01382935   0.1423620   0.43600905
sigma   4.76114350   4.9816850   5.10326000   5.2300450   5.51500975
";

Extract the parameter posterior means: `β`,

In [16]:
[posterior_β, posterior_σ]

2-element Array{Any,1}:
  [154.6231386427075, 5.856041468814334, -0.013078530905223562]
 5.094422864458341                                             

end of m4.5d.jl#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*